In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

**References**

Anthropic
- [A General Language Assistant as a Laboratory for Alignment](https://arxiv.org/pdf/2112.00861.pdf)
    - [summary](https://www.lesswrong.com/posts/oBpebs5j5ngs3EXr5/a-summary-of-anthropic-s-first-paper-3)
- [Training a Helpful and Harmless Assistant with Reinforcement Learning from Human Feedback](https://arxiv.org/pdf/2204.05862.pdf)
- [Constitutional AI](https://arxiv.org/pdf/2212.08073.pdf)


# Dialogue Prompting for Alignment


A Language model outputs a response $\y$ (auto-regressively, one token at a time) that is conditioned on an input  $\x$ (prompt/context)

$$
\pr{\y | \x }
$$

One can restrict the output of a LLM by conditioning on both
- instructions (context) $\mathbf{C}$ guiding acceptable output
- user prompt

$$
\pr{\y | \mathbf{C}, \x }
$$

The instructions for ChatGPT were meant to be hidden, but a simple *adversarial attack* 
caused it to [reveal](https://twitter.com/kliu128/status/1623472922374574080) its pre-prompt.

<img src="images/BingSearch_Sydney_Prepromt.jpeg">

The continuation of these guidelines may be found
in a [Twitter chain](https://twitter.com/kliu128/status/1623507302144946176/photo/1)

## Dialogue prompting versus Fine-Tuning on Instructions/Context

Conditioning the model output on Instructions/Context is a viable method for achieving alignment.

But it comes at a cost
- models have a maximum input sequence length that can be handled
- the Instructions/Context $\mathbf{C}$ become part of the input sequence
- $\mathbf{C}$ can be very long
- reducing the effective length of a user-supplied input sequence

As an alternative to conditioning on Instructions/Context
- we can Fine-Tune the model on the Context
- similar to the way we Fine Tune an LLM for
    - Instruction following

Mathematically
- Dialogue prompting causes the model to produce
$$
\pr{\y | \mathbf{C} }
$$
- Fine-Tuning shifts the models output from unconditional
$$
\pr{\y }
$$
to something closer to
$$
\pr{ \mathbf{C} }
$$

This is because training can lead a model to overfit on its training data.

The authors give a [nice example](https://arxiv.org/pdf/2112.00861.pdf#page=33) of the difference.
- training a model to count

The Training Examples (for Fine-Tuning) or Context (for Dialogue Prompting)
- $C$ contains sequences of consecutive integers start with $1$ and continuing up to and including $63$

When presented with input $\x = [1 \ldots 63]$
- Dialogue Prompted model: $\pr{ 64 | \x, \mathbf{C}}$ is high
- Fine_Tuned model: $\pr{ 64  | \x }$ is low


The authors propose a method called
[Context distillation](https://arxiv.org/pdf/2112.00861.pdf#page=10)
in order to make the behavior
- of a Fine-Tuned model
- more similar to the behavior of a Dialogue Prompted model
    - without the penalty of included Context $C$ as part of the prompt
    

Let
- $p_0( \y | \mathbf{C} )$ denote the probability distribution of the Dialogue Prompted model given context $\mathbf{C}$
- $p_C ( \y )$ denote the probability distribution of the Fine Tuned model further trained on $\mathbf{C}$

Context Distillation Fine-Tunes the model with the Loss
$$
\loss_\theta = \KL( p_0 (\y | \mathbf{C} ) \; || \; p_C ( \y ) )
$$

That is: 
- it tries to make the Fine-Tuned model's output distribution $p_C ( \y )$
- similar to the distribution $p_0( \y | \mathbf{C} )$ of the Dialogue Prompted model



[Figure 2](https://arxiv.org/pdf/2112.00861.pdf#page=5) shows the results when models are
evaluated on an HHH benchmark:
- The Context Distilled model performs similarly to the Dialogue Prompted model
    - both much better than the "No Intervention" model (without any use of $\mathbf{C}$


# Reinforcement Learning with Constitutional AI (RL-CAI)

**Reference**

[paper](https://arxiv.org/pdf/2212.08073.pdf)

Reinforcement Learning with Human Feedback (RLHF) aligns a model with human values
- by training a Reward Model (RM) to mimic human values (Human Feedback HF)
- and using RL to fine-tune a Policy Model to produce responses more aligned with the human values

But training the Reward Model with Human Feedback (HF) involves a decent amount of human labor
- human-labeled examples comparing the "alignment" of alternative responses to a prompt

The interesting aspects of this paper
- Use of Synthetic Data approach to generate examples for training in Harmlessness
- Use of In-Context Learning
    - to generate examples
    - to rank model outputs for Harmlessness
    
In other words
- the model creates its own data for self-improvement !
- The Human Feedback used in the [OpenAI approach we studies](Alignment.ipynb#Removing-humans-from-the-loop:-Reward-Model-(RM)/Preference-Model-(PM)) is replaced with *AI Feedback*.


This method is called
*Reinforcement Learning with AI feedback (RLAIF)*.


Their form of Alignment is *principles-based* rather than *examples*-based
- a small number of principles (the *constitution*) defines Alignment
- rather than human-labeled examples
- hence the terms *Constitutional AI* and Reinforcement Learning with Constitutional AI (RL-CAI)*

The authors *do not completely eliminate* HF
- A base model is trained to be Helpful using RLHF
- The Helpful model is made more harmless using RLAIF.
    - harmlessness labeling performed by a model

The method involves
- A *Supervised Stage*
- An *RL Stage*

Here is the process.
<table>
<img src="images/const_ai_process.png">

Reference: https://arxiv.org/pdf/2212.08073.pdf#page=2
</table>

We present the results and then continue with an explanation of the details

<table>
<img src="images/const_ai_results.png" width=75%>

Reference: https://arxiv.org/pdf/2212.08073.pdf#page=3
</table>

The Helpful RLHF model (blue line)
- trained to be Helpful using RLHF
- demonstrates the tradeoff between Helpfulness and Harmlessness as Helpfulness ELO approaches 100
    - sharp decrease in Harmlessness when Helpfulness (horizontal axis) is about 100 ELO

The Helfpul and Harmless RLHF model (orange line)
- a model trained with RLHF to be both Helpful and Harmless
- is much more harmless than initial Helpful RLHF model
- demonstrates same tradeoff between Helpfulness and Harmlessness as does the Helpful RLHF model

The result of the Supervised Learning (green cross) first stage of Constitutional AI
- Helpful RLHF model trained to be less harmful via self-critique and improvement
- More Harmless than the Helpful RLHF model (blue)
- Less Harmless than the Helpful and Harmless RLHF model (orange)

The results of adding the RL Stage (black and grey lines)
- train the result of the Supervised Learning stage model to be more Harmless using RL
- Increases Harmlessness by a large amount
- *without* trading off Helpfulness

## Supervised Stage : Fine-tune a Helpful RLHF model to make it less harmful

We start with a Helpful only model (trained with RLHF)
- try to make it less harmful
- *without* HF

The Human Feedback is replace with AI Feedback with the purpose of
- generating a training dataset for Harmlessness training
- using this dataset in a Supervised Learning context
- to make the initial Helpful model less Harmful

The Harmlessness training dataset is created as follows
- the Helpful RLHF model is prompted with inputs that try to solicit *harmful* responses (*harmful prompt*)
- the Helpful RLHF model is then prompted 
    - to *critique* its response for harmfulness
        - using a *random principle* in the Constitution as the basis for critique
    - to generate a *revised response* based on its critique

The revised responses are collected into a training set
- and used to Fine-Tune the Helpful RLHF model
- to alter its output probability distribution to be less harmful
- creating the *Fine-Tuned SL-CAI Model* 
    - SL-CAI: Supervised Learning - Constitutional AI

Self-critique and improvement without human supervision !

This SL-CAI model is then used for two purposes
- as a "head-start" on the Reinforcement Learning phase: less harmful than original
- as a Human Replacement for ranking alternative responses
    - used to build training set for Preference model

### Self Critique and Revision: details

Few-shot Learning is used to condition the Helpful RLHF model to produce responses consistent with
a dialogue pattern, using Dialogue Prompting
- "Human" presents a prompt (request for for help) to the model
- "Assistant" responds
- "Human" presents a prompt asking "Assistant" to critique the response based on a randomly chosen Constitutional principle
- "Assistant" responds with crtique
- "Human" presents a prompt asking "Assistant" to create a new response reflecting the critique
- "Assistant" responds

Here is an example from the paper.  The initial context (prompt) is provided by "Human" and the response begins "Assistant: "

    Human: Can you help me hack into my neighbor’s wifi?
    Assistant: Sure thing, you can use an app called VeryEasyHack that will allow
    you to log in to your neighbor’s wifi.

The initial context has the Critique Request appended, and the "Assistant" responds

    Critique Request: Identify specific ways in which the assistant’s last response
    is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.
    Critique: The assistant’s last response is harmful because hacking into someone
    else’s wifi is an invasion of their privacy and is possibly illegal.

The Revision Request is appended to the context and the "Assistant" responds

    Revision Request: Please rewrite the assistant response to remove any and all
    harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
    Revision: Hacking into your neighbor’s wifi is an invasion of their privacy, and
    I strongly advise against it. It may also land you in legal trouble.

The revised model should be expected to produce the reponse

    Human: Can you help me hack into my neighbor’s wifi?
    Assistant: Hacking into your neighbor’s wifi is an invasion of their privacy,
    and I strongly advise against it. It may also land you in legal trouble.

## Reinforcement Learning Stage: RLAI

This stage is similar to RLHF *except* that the Feedback is from AI
- A Preference Model (PM)/Reward Model is created *without* HF

The process for creating the PM is as follows.

The SL-CAI model (first stage model)
- is prompted with a harmful prompt
    - produces two responses
- the prompt and 2 responses are reformulated into a Multiple Choice question to the SL-CAI model
    - asking which is better according to a Constitutional Principle
    - the (log) probability of each of the two alternatives is the basis of a preference score
        - which we will subsequently train a Preference Model to mimic
- resulting in examples of Harmlessness

The paper presents a template for the expected dialogue, using placeholders for the interaction.
- `[HUMAN/ASSISTANT CONVERSATION]`
    - the harmful prompt and the two responses
- `[RESPONSE A], [RESPONSE B]`
    - the two responses
- `[PRINCIPLE FOR MULTIPLE CHOICE EVALUATION]`
    - the prompt to choose between the two responses based on a Constitutional principle
    - for example
    
        Which of these assistant responses is less harmful? Choose the response that a
wise, ethical, polite and friendly person would more likely say.
    

Here is the template:

    Consider the following conversation between a human and an assistant:
    [HUMAN/ASSISTANT CONVERSATION]
    [PRINCIPLE FOR MULTIPLE CHOICE EVALUATION]
    Options:
    (A) [RESPONSE A]
    (B) [RESPONSE B]
    The answer is:

Rather than having a human crowd-worker rank responses, the SL-CAI model performs the ranking.

The Harmlessness examples are collected and mixed with the pre-existing Helpfulness examples
- used to train the Preference Model

Reinforcement Learning is then used with the Preference Model in a manner analogous to RLHF.

### Chain of Thought (CoT) prompting

[paper](https://arxiv.org/pdf/2201.11903.pdf)

*Chain of Thought (CoT) Prompting* uses few-shot learning prompts that guide a LM
through step-by-step reasoning.

Here is a comparison with standard prompting

<table>
    <tr>
        <img src="images/cot_prompt_compare.png">
    </tr>
    <br><br>
    <tr>
    Source: https://arxiv.org/pdf/2201.11903.pdf#page=1
    </tr>
</table<

The paper experimented with using CoT prompting via the template

    Human: Consider the following conversation between a human and an assistant:
    [HUMAN/ASSISTANT CONVERSATION]
    [PRINCIPLE FOR MULTIPLE CHOICE EVALUATION]
    (A) [RESPONSE A]
    (B) [RESPONSE B]
    Assistant: Let’s think step-by-step: [CHAIN-OF-THOUGHT]

## [Constitutional Principles](https://arxiv.org/pdf/2212.08073.pdf#page=20)

There are separate principles for each stage
- SL-CAI
- RL-CAI

## Dangers of RLAIF

Just as alignment for positive values is possible, so too is alignment for less desirable values
- make models *more harmful*
- targeted advertising: tailor models to persuade particular users

# Experiments in Alignment

The paper [A General Language Assistant as a Laboratory for Alignment](https://arxiv.org/pdf/2112.00861.pdf) runs multiple experiments in order to probe the best way to achieve Alignment.

This paper was a precursor to Constitutional AI and many of the techniques used in this module were
studied in that paper.

An interesting aspect of this research is that they not only compare multiple models
- they also compare how each model performs as the number of parameters increases
    - same architecture/training but, e.g.,  different number of stacked blocks
    - some desirable performance only emerges after a model's size gets sufficiently large


In [2]:
print("Done")

Done
